# Operaciones básicas sqlite3

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from sqlalchemy import create_engine

In [60]:
con = sqlite3.connect('example.db')
cur = con.cursor()

In [4]:
# Create table
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')

cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
cur.execute("INSERT INTO stocks VALUES ('2005-01-08','SELL','RHAT',200,70.99)")

In [13]:
for row in cur.execute('SELECT * FROM stocks ORDER BY date'):
        print(row)

('2005-01-08', 'SELL', 'RHAT', 200.0, 70.99)
('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [8]:
# Save (commit) the changes
con.commit()
con.close()

In [46]:
headers = tuple([i[1] for i in cur.execute("PRAGMA table_info(stocks)")])

In [25]:
print(headers)
for row in cur.execute('SELECT * FROM stocks ORDER BY date'):
        print(row)

('date', 'trans', 'symbol', 'qty', 'price')
('2005-01-08', 'SELL', 'RHAT', 200.0, 70.99)
('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [26]:
from tabulate import tabulate

In [61]:
table =   [row for row in cur.execute('SELECT * FROM stocks')]
headers = [field[1] for field in cur.execute("PRAGMA table_info(stocks)")]

print(tabulate(table,headers,tablefmt="fancy_grid"))

╒════════════╤═════════╤══════════╤═══════╤═════════╤══════════╕
│ date       │ trans   │ symbol   │   qty │   price │ totals   │
╞════════════╪═════════╪══════════╪═══════╪═════════╪══════════╡
│ 2006-01-05 │ BUY     │ RHAT     │   100 │   35.14 │          │
├────────────┼─────────┼──────────┼───────┼─────────┼──────────┤
│ 2005-01-08 │ SELL    │ RHAT     │   200 │   70.99 │          │
╘════════════╧═════════╧══════════╧═══════╧═════════╧══════════╛


In [44]:
for row in cur.execute("SELECT SUM(price) FROM stocks"):
    print(row)

(106.13,)


In [41]:
for row in cur.execute('SELECT qty FROM stocks'):
        print(row)

['date', 'trans', 'symbol', 'qty', 'price']
(100.0,)
(200.0,)


In [48]:
cur.execute("ALTER TABLE stocks ADD totals real")

In [57]:
print_db()

╒════════════╤═════════╤══════════╤═══════╤═════════╤══════════╕
│ date       │ trans   │ symbol   │   qty │   price │ totals   │
╞════════════╪═════════╪══════════╪═══════╪═════════╪══════════╡
│ 2005-01-08 │ SELL    │ RHAT     │   200 │   70.99 │          │
├────────────┼─────────┼──────────┼───────┼─────────┼──────────┤
│ 2006-01-05 │ BUY     │ RHAT     │   100 │   35.14 │          │
╘════════════╧═════════╧══════════╧═══════╧═════════╧══════════╛


# Crear un pandas, y de él un db

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from tabulate import tabulate
from sqlalchemy import create_engine

In [2]:
df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])

In [3]:
df

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [4]:
# Guardado de pandas a db

engine = create_engine('sqlite:///pandas2db.db', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "mytable"
df.to_sql(sqlite_table, sqlite_connection, if_exists='fail', index=False)

2021-12-04 23:21:13,384 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("mytable")
2021-12-04 23:21:13,386 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-04 23:21:13,387 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("mytable")
2021-12-04 23:21:13,389 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-04 23:21:13,390 INFO sqlalchemy.engine.Engine 
CREATE TABLE mytable (
	a INTEGER, 
	b INTEGER, 
	c INTEGER
)


2021-12-04 23:21:13,391 INFO sqlalchemy.engine.Engine [no key 0.00047s] ()
2021-12-04 23:21:13,402 INFO sqlalchemy.engine.Engine COMMIT
2021-12-04 23:21:13,403 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-04 23:21:13,404 INFO sqlalchemy.engine.Engine INSERT INTO mytable (a, b, c) VALUES (?, ?, ?)
2021-12-04 23:21:13,404 INFO sqlalchemy.engine.Engine [generated in 0.00042s] ((1, 2, 3), (4, 5, 6), (7, 8, 9))
2021-12-04 23:21:13,406 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
# Cargado de db

con = sqlite3.connect('pandas2db.db')
cur = con.cursor()

table =   [row for row in cur.execute('SELECT * FROM mytable')]
headers = [field[1] for field in cur.execute("PRAGMA table_info(mytable)")]

print(tabulate(table,headers,tablefmt="fancy_grid"))

╒═════╤═════╤═════╕
│   a │   b │   c │
╞═════╪═════╪═════╡
│   1 │   2 │   3 │
├─────┼─────┼─────┤
│   4 │   5 │   6 │
├─────┼─────┼─────┤
│   7 │   8 │   9 │
╘═════╧═════╧═════╛


In [6]:
# De db a pandas

df_db = pd.read_sql_query("SELECT * FROM mytable", con)

In [7]:
df_db

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9
